# 1. Least Angle Regression

In [55]:
import numpy as np
import plotly.graph_objects as go
from sklearn import linear_model
from sklearn import datasets

### Data Preparation

In [56]:
diabetes = datasets.load_diabetes()
x = diabetes.data
y_not_standardized = diabetes.target

x is already standardized. however y is not.

In [57]:
y = y_not_standardized - y_not_standardized.mean()
print(y_not_standardized.mean())
print(y.mean())

152.13348416289594
-7.716301202824617e-15


### lars function

In [58]:
def lars(x, y, tol=1e-10):
    # initialization
    n = x.shape[0]
    m = x.shape[1]
    beta = np.zeros(m)
    prediction = np.zeros(n)
    active_set = []
    signs = np.zeros(m)
    tol=1e-8
    beta_path = [beta.copy()] 
    prediction_path = [prediction.copy()]
    active_sets = [active_set.copy()]

    for step in range(m):
        residual = y - prediction    
        correlations = x.T @ residual
        c_max = np.max(np.abs(correlations))
        if c_max < tol:
            break
        candidates = np.where(np.abs(np.abs(correlations)-c_max) <= tol)[0] # can i do this better?
        for j in candidates:
            if j not in active_set:
                active_set.append(int(j))
                signs[j] = np.sign(correlations[j]) if correlations[j] != 0.0 else 1.0 # check this
        nump_active_set = np.array(active_set, dtype=int) # consider adding dype elsewhere
        active_signs = signs[nump_active_set]
        active_x = x[:, nump_active_set] * active_signs
        g_a = active_x.T @ active_x
        g_a_inverse = np.linalg.inv(g_a)
        ones_a = np.ones(len(nump_active_set))
        a_a = 1.0 / np.sqrt(ones_a.T @ g_a_inverse @ ones_a)
        w_a = a_a * (g_a_inverse @ ones_a)
        u_a = active_x @ w_a
        little_a = x.T @ u_a
        
        #find gamma step size
        gamma = np.inf
        j_star = None
        inactive = [j for j in range(m) if j not in active_set]
        if not inactive:
            break
        for j in inactive:
            denominator_1 = a_a - little_a[j]
            if denominator_1 > tol:
                gamma1 = (c_max - correlations[j]) / denominator_1
                if gamma1 > tol and gamma1 < gamma:
                    gamma = gamma1
                    j_star = j
            denominator_2 = a_a + little_a[j]
            if denominator_2 > tol:
                gamma2 = (c_max + correlations[j]) / denominator_2
                if gamma2 > tol and gamma2 < gamma:
                    gamma = gamma2
                    j_star = j
        
        # update prediction and beta
        prediction += gamma * u_a
        for index, j in enumerate(nump_active_set):
            beta[j] += gamma * w_a[index] * signs[j]
        if j_star not in active_set: # do i need this condition?
            active_set.append(j_star)
            signs[j_star] = np.sign(correlations[j_star]) if correlations[j_star] != 0.0 else 1.0

        beta_path.append(beta.copy())
        prediction_path.append(prediction.copy())
        active_sets.append(active_set.copy())


    beta_path = np.vstack(beta_path)
    prediction_path = np.vstack(prediction_path)


    return beta_path, prediction_path, active_sets

In [59]:
beta_path, prediction_path, active_sets = lars(x, y)

# Plot results

In [ ]:
K_plus_1, m = beta_path.shape
n = x.shape[0]

l1_norm = np.sum(np.abs(beta_path), axis=1)

residual_path = y[None, :] - prediction_path
corr_path = residual_path @ x
abs_corr_path = np.abs(corr_path)
C_k = np.max(abs_corr_path, axis=1)
steps = np.arange(K_plus_1)

In [41]:
import plotly.graph_objects as go

fig_coeff = go.Figure()

for j in range(m):
    fig_coeff.add_trace(
        go.Scatter(
            x=l1_norm,
            y=beta_path[:, j],
            mode="lines",
            name=f"β{j+1}",
            showlegend=False
        )
    )

fig_coeff.update_layout(
    template="simple_white",
    font=dict(
        family="DejaVu Sans",
        size=18,
        color="black"
    ),
    margin=dict(l=60, r=20, t=40, b=50),
    width=600,
    height=600,
)

fig_coeff.update_xaxes(
    title_text="∑ |β̂_j| →",
    showline=True, linecolor="black", linewidth=1,
    mirror=True,
    ticks="outside",
    showgrid=False,
)
fig_coeff.update_yaxes(
    title_text="β̂_j",
    showline=True, linecolor="black", linewidth=1,
    mirror=True,
    ticks="outside",
    showgrid=False,
)

fig_coeff.show()


# Figure 2
fig_corr = go.Figure()

for j in range(m):
    fig_corr.add_trace(
        go.Scatter(
            x=steps,
            y=abs_corr_path[:, j],
            mode="lines",
            line=dict(width=1),
            showlegend=False
        )
    )

fig_corr.add_trace(
    go.Scatter(
        x=steps,
        y=C_k,
        mode="lines+markers",
        line=dict(width=3),
        name="Ĉ_k"
    )
)

fig_corr.update_layout(
    template="simple_white",
    font=dict(
        family="DejaVu Sans",
        size=18,
        color="black"
    ),
    margin=dict(l=60, r=20, t=40, b=50),
    width=600,
    height=600
)

fig_corr.update_xaxes(
    title_text="Step k →",
    showline=True, linecolor="black", linewidth=1,
    mirror=True,
    ticks="outside",
    showgrid=False,
    dtick=2,
)
fig_corr.update_yaxes(
    title_text="|ĉ_kj|",
    showline=True, linecolor="black", linewidth=1,
    mirror=True,
    ticks="outside",
    showgrid=False,
)

fig_corr.show()


# 2. Generalized Heron's Method

Heron's method appoximate the value of square roots with the formula
$$x_{n+1} = \frac{1}{2}\left( x_n + \frac{S}{x_n} \right)$$

In [42]:
def heron_method(s, x0=1.0, tol=1e-10, max_iter=100):
    x_n = x0
    for n in range(max_iter):
        x_n1 = 0.5 * (x_n + s / x_n)
        if abs(x_n1 - x_n) < tol:
            return x_n1
        x_n = x_n1
    return x_n

We can generalize Heron's method using

99999.99999499999